In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import os

from keras.preprocessing import sequence
np.random.seed(7)
from keras.utils import pad_sequences

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-04-13 09:40:20.832168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 09:40:27.635633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sanju/ns-allinone-2.35/otcl-1.14:/home/sanju/ns-allinone-2.35/lib
2023-04-13 09:40:27.635728: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-13 09:40:45.494761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7';

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15945303449841297456
xla_global_id: -1
]


2023-04-13 09:40:59.576629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 09:40:59.821739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-04-13 09:40:59.821890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sanju-HP-Notebook): /proc/driver/nvidia/version does not exist


In [2]:
X=[]
y=[]
base_path='dataset/'
source_path=base_path
for child in os.listdir(source_path):
    sub_path = os.path.join(source_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path):
            file_path = os.path.join(sub_path, data_file)
            print(file_path)
            data=pd.read_csv(file_path,header=None)
            data=data.to_numpy()
            X.append(data)
            y.append(child)           

dataset/1/8.csv
dataset/1/5.csv
dataset/1/10.csv
dataset/1/9.csv
dataset/1/1.csv
dataset/1/3.csv
dataset/1/6.csv
dataset/1/7.csv
dataset/1/2.csv
dataset/1/4.csv
dataset/1/11.csv
dataset/1/12.csv
dataset/2/5.csv
dataset/2/13.csv
dataset/2/14.csv
dataset/2/9.csv
dataset/2/17.csv
dataset/2/1.csv
dataset/2/3.csv
dataset/2/6.csv
dataset/2/7.csv
dataset/2/16.csv
dataset/2/2.csv
dataset/2/15.csv
dataset/2/4.csv
dataset/2/11.csv
dataset/2/12.csv


In [12]:
max_review_length = 100
X_tr = pad_sequences(X, maxlen=max_review_length)

In [13]:
print(X)

[array([[            nan,  0.00000000e+00,  1.00000000e+00, ...,
         6.00000000e+01,  6.10000000e+01,  6.20000000e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33959949e-02, -4.29150671e-01, -9.71492807e-02],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -7.00426102e-03, -4.49372113e-01, -8.56205374e-02],
       ...,
       [ 2.60000000e+01,  0.00000000e+00,  0.00000000e+00, ...,
         1.55104697e-02, -2.00954735e-01, -2.76311963e-02],
       [ 2.70000000e+01,  0.00000000e+00,  0.00000000e+00, ...,
         1.68442726e-02, -2.02204227e-01, -2.98058847e-02],
       [ 2.80000000e+01,  0.00000000e+00,  0.00000000e+00, ...,
         1.80829465e-02, -2.16355979e-01, -3.81522928e-02]]), array([[            nan,  0.00000000e+00,  1.00000000e+00, ...,
         6.00000000e+01,  6.10000000e+01,  6.20000000e+01],
       [ 0.00000000e+00,             nan,             nan, ...,
                    nan,             nan,            

In [14]:
print(y)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]]


In [15]:
print(X_tr)

[[[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [ 26   0   0 ...   0   0   0]
  [ 27   0   0 ...   0   0   0]
  [ 28   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [ 55   0   0 ...   0   0   0]
  [ 56   0   0 ...   0   0   0]
  [ 57   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [ 63   0   0 ...   0   0   0]
  [ 64   0   0 ...   0   0   0]
  [ 65   0   0 ...   0   0   0]]

 ...

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [ 79   0   0 ...   0   0   0]
  [ 80   0   0 ...   0   0   0]
  [ 81   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [ 81   0   0 ...   0   0   0]
  [ 82   0   0 ...   0   0   0]
  [ 83   0   0 ...   0   0  

In [16]:
X_tr=np.array(X_tr)
y=np.array(y)

In [17]:
y = np.asarray(y).astype('float32').reshape((-1,1))

In [18]:
modelClass = Sequential()
modelClass.add(LSTM(100, return_sequences=True, input_shape=(100,64)))
modelClass.add(LSTM(50, return_sequences=True, activation='relu'))
modelClass.add(LSTM(20, return_sequences=True, activation='relu'))
modelClass.add(LSTM(10, activation='relu'))
modelClass.add(Dense(1, activation='sigmoid'))
modelClass.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelClass.summary())
modelClass.fit(X_tr, y, epochs=30, batch_size=16)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100, 100)          66000     
                                                                 
 lstm_9 (LSTM)               (None, 100, 50)           30200     
                                                                 
 lstm_10 (LSTM)              (None, 100, 20)           5680      
                                                                 
 lstm_11 (LSTM)              (None, 10)                1240      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 103,131
Trainable params: 103,131
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
2/2 [===========================

In [11]:
scores = modelClass.evaluate(X_tr, y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 44.44%
